In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-03 17:47:16,435 INFO: Initializing external client
2025-03-03 17:47:16,441 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 17:47:18,695 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665
Fetching data from 2025-02-02 22:47:16.425365+00:00 to 2025-03-03 21:47:16.425365+00:00
2025-03-03 17:48:15,596 ERROR: Flight returned timeout error, with message: Deadline Exceeded
Traceback (most recent call last):
  File "c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 209, in __init__
    self._health_check()
  File "c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packages\retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packages\retrying.py", line 266, in call
    raise attempt.get()
          ^^^^^^^^^^^^^
  File "c:\Users\Yas

2025-03-03 17:48:22,770 ERROR: Flight returned timeout error, with message: Deadline Exceeded
Traceback (most recent call last):
  File "c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 394, in afs_error_handler_wrapper
    return func(instance, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 459, in read_query
    return self._get_dataset(
           ^^^^^^^^^^^^^^^^^^
  File "c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packages\retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packages\retrying.py", line 257, in call
    return attempt.get(self._wrap_exception)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packages\retrying.py", line 3

AttributeError: 'ArrowFlightClient' object has no attribute '_server_version'

In [ ]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-02 23:00:00+00:00,2,0
1,2025-02-03 00:00:00+00:00,2,0
2,2025-02-03 01:00:00+00:00,2,0
3,2025-02-03 02:00:00+00:00,2,0
4,2025-02-03 03:00:00+00:00,2,0
...,...,...,...
174189,2025-03-03 16:00:00+00:00,263,89
174190,2025-03-03 17:00:00+00:00,263,114
174191,2025-03-03 18:00:00+00:00,263,116
174192,2025-03-03 19:00:00+00:00,263,102


In [ ]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174194 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174194 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174194 non-null  int32                  
 2   rides               174194 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [ ]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [ ]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174194 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174194 non-null  datetime64[us]
 1   pickup_location_id  174194 non-null  int32         
 2   rides               174194 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [ ]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [ ]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,32,48,92,76,8,155,62,9,60,80,...,5,21,120,218,146,111,120,82,263,2025-02-21 14:00:00
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,210,2025-02-26 19:00:00
2,3,0,0,0,0,2,0,2,0,1,...,3,0,2,1,2,1,1,1,216,2025-02-18 22:00:00
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,147,2025-02-14 01:00:00
4,3,2,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,217,2025-03-01 16:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,67,2025-02-11 03:00:00
247,0,0,0,0,0,0,4,0,1,0,...,0,1,0,0,0,1,1,0,86,2025-02-26 15:00:00
248,0,0,1,0,0,0,2,2,0,1,...,0,1,3,0,0,0,0,2,62,2025-02-28 07:00:00
249,243,130,148,572,217,231,173,141,160,338,...,10,0,133,311,225,168,179,37,142,2025-02-24 02:00:00


In [ ]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-03 17:41:49,723 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 17:41:49,814 INFO: Initializing external client
2025-03-03 17:41:49,819 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 17:41:51,674 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665
Fetching data from 2025-02-02 22:41:49.722798+00:00 to 2025-03-03 21:41:49.722798+00:00
2025-03-03 17:42:49,054 ERROR: Flight returned timeout error, with message: Deadline Exceeded
Traceback (most recent call last):
  File "c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 209, in __init__
    self._health_check()
  File "c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packages\retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packag

2025-03-03 17:42:56,934 ERROR: Flight returned timeout error, with message: Deadline Exceeded
Traceback (most recent call last):
  File "c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 394, in afs_error_handler_wrapper
    return func(instance, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 459, in read_query
    return self._get_dataset(
           ^^^^^^^^^^^^^^^^^^
  File "c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packages\retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packages\retrying.py", line 257, in call
    return attempt.get(self._wrap_exception)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packages\retrying.py", line 3

AttributeError: 'ArrowFlightClient' object has no attribute '_server_version'

In [ ]:
current_date

Timestamp('2025-02-13 22:37:17.628670+0000', tz='Etc/UTC')

In [ ]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,2,2025-02-12 23:00:00
1,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,3,2025-02-12 23:00:00
2,0,0,0,0,0,0,0,5,5,8,...,4,5,6,10,5,5,9,5,4,2025-02-12 23:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,2025-02-12 23:00:00
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-02-12 23:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,0,0,0,0,0,0,0,3,0,0,...,0,0,0,0,1,0,1,0,259,2025-02-12 23:00:00
252,0,0,0,0,1,0,0,0,2,3,...,3,1,1,1,0,4,0,1,260,2025-02-12 23:00:00
253,0,0,0,0,0,0,4,2,6,12,...,36,27,50,60,41,27,30,17,261,2025-02-12 23:00:00
254,0,0,0,0,2,4,14,75,138,162,...,126,90,127,162,96,56,47,22,262,2025-02-12 23:00:00


In [ ]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-02-13 17:41:32,318 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-13 17:41:32,334 INFO: Initializing external client
2025-02-13 17:41:32,337 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-13 17:41:33,715 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211551


In [ ]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [ ]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,2.0
3,5,0.0
4,6,0.0
...,...,...
251,259,0.0
252,260,1.0
253,261,10.0
254,262,10.0


In [ ]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([132, 249, 230, 161,  79, 142,  48, 114, 138, 163])